 ## <center> Close Price Prediction

The system offers live data pulling functionality with 10+ features for predicting daily close price of any crypto, the process is pretty simple just change the symbol and run the process for any currency  


In [1]:
import pandas as pd
import numpy as np
import ssl
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 20,10
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM

In [2]:
symbol = 'ETHUSDT'
filepath = f"https://www.cryptodatadownload.com/cdd/Binance_{symbol}_d.csv"  
ssl._create_default_https_context = ssl._create_unverified_context

df = pd.read_csv(filepath, skiprows=1)
df.head() 

,unix,date,symbol,open,high,low,close,Volume ETH,Volume USDT,tradecount
0,1.664669e+12,2022-10-02 00:00:00,ETH/USDT,1311.79,1313.00,1310.43,1312.18,2119.7334,2.780693e+06,2637.0
1,1.664582e+12,2022-10-01 00:00:00,ETH/USDT,1328.71,1333.69,1302.68,1311.79,252081.9056,3.329301e+08,347396.0
2,1.664496e+12,2022-09-30 00:00:00,ETH/USDT,1335.69,1373.19,1313.35,1328.72,740753.6110,9.915054e+08,926935.0
3,1.664410e+12,2022-09-29 00:00:00,ETH/USDT,1337.20,1352.02,1288.52,1335.70,770591.6895,1.022674e+09,950476.0
4,1.664323e+12,2022-09-28 00:00:00,ETH/USDT,1328.01,1355.70,1253.20,1337.20,919314.9131,1.199743e+09,1075441.0


In [ ]:
#setting index as date
df['date'] = pd.to_datetime(df.date,format='%Y-%m-%d')
df.index = df['date']

#plot
plt.figure(figsize=(16,8))
plt.plot(df['close'], label='Close Price history')

In [ ]:
#creating dataframe
data = df.sort_index(ascending=True, axis=0)
new_data = pd.DataFrame(index=range(0,len(df))
                        ,columns=['date', 'close'])
for i in range(0,len(data)):
    new_data['date'][i] = data['date'][i]
    new_data['close'][i] = data['close'][i]

#setting index
new_data.index = new_data.date
new_data.drop('date', axis=1, inplace=True)    

In [ ]:
#creating train and test sets
dataset = new_data.values

train = dataset[0:1000,:]
valid = dataset[1000:,:]

#converting dataset into x_train and y_train
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset) 

In [ ]:
x_train, y_train = [], []
for i in range(120,len(train)):
    x_train.append(scaled_data[i-120:i,0])
    y_train.append(scaled_data[i,0])
    
x_train, y_train = np.array(x_train), np.array(y_train)

x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))

In [ ]:
x_train.shape

In [ ]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(units=50, return_sequences=True,
               input_shape=(x_train.shape[1],1)))
model.add(LSTM(units=50))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')
history = model.fit(x_train, y_train, epochs=4, batch_size=1, verbose=1)

In [ ]:
model.summary()

In [ ]:
plt.plot(np.array([1, 2, 3,4]), history.history['loss'], color='r', label='Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
inputs = new_data[len(new_data) - len(valid) - 120:].values
inputs = inputs.reshape(-1,1)
inputs  = scaler.transform(inputs)

X_test = []
for i in range(120,inputs.shape[0]):
    X_test.append(inputs[i-120:i,0])
X_test = np.array(X_test)

X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))

closing_price = model.predict(X_test)
closing_price = scaler.inverse_transform(closing_price)

In [ ]:
#for plotting
plt.figure(figsize=(16,8))
train = new_data[:1000]
valid = new_data[1000:]
valid['Predictions'] = closing_price
plt.plot(train['close'], label='Market Previous' )
plt.plot(valid['close'], label='Real Price' )
plt.plot(valid['Predictions'], label='Predicted Price' )
plt.legend()
plt.show()

In [ ]:
def PredictTodayClose():
  Last120DaysPrices = new_data.values
  Last120DaysPrices = Last120DaysPrices[Last120DaysPrices.shape[0]-121:-1]
  Last120DaysPrices = Last120DaysPrices.reshape(-1,1)
  Last120DaysPrices = scaler.transform(Last120DaysPrices)
  Last120DaysPrices = np.reshape(Last120DaysPrices, (1,Last120DaysPrices.shape[0], 1))
  predicted = model.predict(Last120DaysPrices)
  predicted = scaler.inverse_transform(predicted)
  return predicted[0][0]

In [ ]:
print(f"My Prediction For Today's ETH Close Price is : {PredictTodayClose()} USDT")